In [1]:
import numpy as np
import collections
import itertools
from sklearn.ensemble import RandomForestClassifier

# Day 4: Passport Processing
You arrive at the airport only to realize that you grabbed your North Pole Credentials instead of your passport. While these documents are extremely similar, North Pole Credentials aren't issued by a country and therefore aren't actually valid documentation for travel in most of the world.

It seems like you're not the only one having problems, though; a very long line has formed for the automatic passport scanners, and the delay could upset your travel itinerary.

Due to some questionable network security, you realize you might be able to solve both of these problems at the same time.

The automatic passport scanners are slow because they're having trouble detecting which passports have all required fields. The expected fields are as follows:

```
byr (Birth Year)
iyr (Issue Year)
eyr (Expiration Year)
hgt (Height)
hcl (Hair Color)
ecl (Eye Color)
pid (Passport ID)
cid (Country ID)
```

Passport data is validated in batch files (your puzzle input). Each passport is represented as a sequence of key:value pairs separated by spaces or newlines. Passports are separated by blank lines.

Here is an example batch file containing four passports:

```
ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
```

The first passport is valid - all eight fields are present. The second passport is invalid - it is missing hgt (the Height field).

The third passport is interesting; the only missing field is cid, so it looks like data from North Pole Credentials, not a passport at all! Surely, nobody would mind if you made the system temporarily ignore missing cid fields. Treat this "passport" as valid.

The fourth passport is missing two fields, cid and byr. Missing cid is fine, but missing any other field is not, so this passport is invalid.

According to the above rules, your improved system would report 2 valid passports.

Count the number of valid passports - those that have all required fields. Treat cid as optional. In your batch file, how many passports are valid?

In [2]:
def hot_encode(keys):
    # ['byr', 'cid', 'ecl', 'eyr', 'hcl', 'hgt', 'iyr', 'pid']
    he = np.array([0, 0, 0, 0, 0, 0, 0, 0])
    if 'byr' in keys:
        he += [1, 0, 0, 0, 0, 0, 0, 0]
    if 'cid' in keys:
        he += [0, 1, 0, 0, 0, 0, 0, 0]
    if 'ecl' in keys:
        he += [0, 0, 1, 0, 0, 0, 0, 0]
    if 'eyr' in keys:
        he += [0, 0, 0, 1, 0, 0, 0, 0]
    if 'hcl' in keys:
        he += [0, 0, 0, 0, 1, 0, 0, 0]
    if 'hgt' in keys:
        he += [0, 0, 0, 0, 0, 1, 0, 0]
    if 'iyr' in keys:
        he += [0, 0, 0, 0, 0, 0, 1, 0]
    if 'pid' in keys:
        he += [0, 0, 0, 0, 0, 0, 0, 1]
    return he

def read_credentials_from_file(filname):
    creds = []
    with open(filname, 'r') as f:
        cred = {}
        for line in f:
            if line == '\n':
                creds.append(hot_encode(dict(sorted(cred.items())).keys()))
                cred = {}
                continue
            
            kv = [vals.split(':') for vals in line.strip().split(' ')]
            for key, value in kv:
                cred[key] = value
    return creds
    
creds = read_credentials_from_file('input/d4.dat')
print(creds[:1])

[array([1, 0, 0, 1, 1, 1, 1, 1])]


In [3]:
# prepare training data

X_train = np.array(list(itertools.product([0, 1], repeat=8)))
y_train = np.array([False] * X_train.shape[0])

idx_1 = np.all(X_train == [1, 1, 1, 1, 1, 1, 1, 1], axis=1)
y_train[idx_1] = True
idx_2 = np.all(X_train == [1, 0, 1, 1, 1, 1, 1, 1], axis=1)
y_train[idx_2] = True

In [4]:
clf = RandomForestClassifier(n_estimators=1, bootstrap=False)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, n_estimators=1)

In [5]:
prediction = clf.predict(creds)
prediction.sum()

228